# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

## 1) Short-Step Path-Following 

In [284]:
using LinearAlgebra
using LaTeXStrings
using DataFrames

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260


In [374]:
"""
           STPF1(c, A, b, init_vect, max_iter, tol_step_x, theta)
       Compute the optimisation of the linear problem min `c`'x, s.t. `A`x = `b` (Primal), 
        with the Short-Step Path Following method.
        
        We have the Dual problem: max `b`'λ, s.t. `A`'λ + s = c.

        `init_vect` shall be the initial vector ``(x_0,λ_0, s_0)``.
        
        `max_iter` is the maximum number of iterations.

        `tol_step_x` is the minimum tolerance of the difference between two successive iterates of x.

        `theta` is the parameter such that ``||XSe- μe||_2 ≤ theta*μ ``, 

            where μ = s'x/length(x), X = Diagonal(x), S = Diagonal(s), and e = ones(1, length(x)).
       """
function STPF1(c, A, b, init_vect, max_iter, tol_step_x= 1e-4, theta=0.4)
    x_k, lambda_k, s_k = init_vect
    n_rows, n_cols = size(A) 
    sigma = 1 - theta / sqrt(n_cols)
    c = 1 # pour iterer
    step_x = 1
    while c <= max_iter && step_x > tol_step_x
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        e = ones(n_cols, 1)
        mu_k = s_k' * x_k / n_cols
        
        # Newton's method:
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [zeros(n_rows+n_cols, 1)
              -Xk*Sk*e + sigma*mu_k*e]
        dir_k = Jacob_Fk\Fk # error if Jacob_Fk is singular
        
        #update:
        x_k_prec = copy(x_k)
        x_k += dir_k[1:n_cols]
        lambda_k += dir_k[n_cols+1: n_cols+n_rows]
        s_k += dir_k[n_cols+n_rows+1:end]
        
        step_x = norm(x_k - x_k_prec)
        c += 1
    end
    
    return Dict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, "n_iter" => c)
end

function display_results(result)
    # fonction pour l'affichage
    for (k,v) in result
        println(k," = ", v)
    end
    
end

    

display_results (generic function with 1 method)

In [375]:
### probleme 1
c = [1, 0 ,0]
A = [1, 1, 1]'
b = [1];

In [376]:
# initialisation 1 
x01 = [1/3, 1/3, 1/3]
lambda01 = [-1]
s01 = [2, 1, 1]
init_vect = (x01, lambda01, s01);
res_opt1 = STPF1(c, A, b, init_vect1, 100);
display_results(res_opt1)

n_iter = 30
x_opt = [0.9997809118396458, 0.00010954408017718925, 0.00010954408017718925]
lambda_opt = [1.000109576843052]
s_opt = [-0.0001095768430525987, -1.000109576843052, -1.000109576843052]


In [322]:
# initialisation 2 
x02 = [1/3, 1/3, 1/3]
lambda02 = [0]
s02 = [1, 0, 0]
init_vect2 = (x02, lambda02, s02);
res_opt2 = STPF1(c, A, b, init_vect2, 100); # Jacobienne non inversible

SingularException: SingularException(0)